In [3]:
import cv2
import mediapipe as mp

In [4]:
mphands=mp.solutions.hands
hands=mphands.Hands()
mpDraw=mp.solutions.drawing_utils
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [5]:
def facedata(image):
    fwidth=0
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    face=face_cascade.detectMultiScale(gray)
    wid=[]
    for x,y,w,h in face:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        wid.append(w)
        fwidth=max(wid)
    return fwidth                             #calculation pixel face width from ref image

In [6]:
kd=65  #real distance from camera to face in cm
rw=15  #real face width

In [7]:
def focal(facew,kdist,rwidth):
    focal=facew*kdist/rwidth
    return focal          #focal length used to calculate real time distance    

In [8]:
def distance(f,facew,rwidth):                   #Real time distance calculation
    distance=rwidth*f/facew
    return distance

In [9]:
reffwidth=361
focallength=1564.33

In [10]:
# refimg=cv2.imread('ref.jpg')
# reffwidth=facedata(refimg)
# focallength=focal(reffwidth,kd,rw)
# print(reffwidth)
# print(focallength)

In [11]:
def main():    #main camera without any modes. m for manual selfie mode. a for auto selfie mode. p for pose detection mode
    a=0          #escape for exit
    m=0
    p=0
    f=0
    while True:
        pressedKey = cv2.waitKey(1) & 0xFF
        flag,frame=cap.read()
        frame=cv2.flip(frame,5)
        if pressedKey == ord('m'):
            m=m+1
            break
        if pressedKey == ord('a'):
            a=a+1
            break
        if pressedKey == 27:
            break
        if pressedKey==ord('p'):
            p=p+1
            break
        if pressedKey==ord('f'):
            f=f+1
            break
        cv2.imshow('cam',frame)
    if m>0:
        return mselfie()
    elif a>0:
        return aselfie()
    elif p>0:
        return pose()
    elif f>0:
        return filtr()
    else:
        cap.release()
        cv2.destroyAllWindows()

In [12]:
def mselfie():          #if distance less than 100 cm press s for snapshot
    while True:
        pressedKey = cv2.waitKey(1) & 0xFF
        flag,frame=cap.read()
        frame=cv2.flip(frame,5)
        framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        facewidth=facedata(framergb)
        if facewidth!=0:
            Distance=distance(focallength,facewidth,rw)
            cv2.putText(frame, f"Distance: {round(Distance,2)} CM", (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        if Distance>100.00:
            cv2.putText(frame, 'please get close to camera', (30, 100), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        if Distance<100.00:
            cv2.putText(frame, 'Press s for snapshot', (30, 100), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
            if pressedKey == ord('s'):
                cv2.imwrite('img1.jpg',frame)
        if pressedKey == ord('x'):
            break
        cv2.imshow('cam',frame)
    return main() 

In [13]:
def aselfie():              #same as above but automatic process
    while True:
        pressedKey = cv2.waitKey(1) & 0xFF
        flag,frame=cap.read()
        frame=cv2.flip(frame,5)
        framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        facewidth=facedata(framergb)
        if facewidth!=0:
            Distance=distance(focallength,facewidth,rw)
            cv2.putText(frame, f"Distance: {round(Distance,2)} CM", (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        if Distance>100.00:
            cv2.putText(frame, 'please get close to camera', (30, 100), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        if Distance<100.00:
                cv2.imwrite('img1.jpg',frame)
                break
        if pressedKey == ord('x'):
            break
        cv2.imshow('cam',frame)
    return main() 

In [14]:
def pose():             #make peace pose and press p for snapshot and make yo pose and press y for snapshot
    while True:
        pressedKey = cv2.waitKey(1) & 0xFF
        flag,frame=cap.read()
        frame=cv2.flip(frame,5)
        framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results=hands.process(framergb)
        p=[0,0,0,0,0]
        f=[]
        if results.multi_hand_landmarks:
            for handlandmark in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame,handlandmark,mphands.HAND_CONNECTIONS)
                for i in handlandmark.landmark:
                    f.append(i)
            if f[8].y<f[6].y:
                p[1]=1
            if f[12].y<f[10].y:
                p[2]=1
            if f[20].y<f[18].y:
                p[4]=1
            if p[1]==1 & p[2]==1:
                if pressedKey == ord('p'):
                    cv2.imwrite('peace.jpg',frame)
            elif p[1]==1 & p[4]==1:
                if pressedKey == ord('y'):
                    cv2.imwrite('yo.jpg',frame)
        if pressedKey == ord('x'):
            break
        cv2.putText(frame, 'pose', (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        cv2.imshow('cam',frame)

    return main()

In [15]:
def filtr():
    face_cascade= cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    while True:
        pressedKey = cv2.waitKey(1) & 0xFF
        flag,frame = cap.read()
        frame=cv2.flip(frame,5)
        #cv2.imshow('main frame',frame)
        #hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        detected_face=[]
        blurred_frame=cv2.GaussianBlur(frame,(25,25),0)
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        if len(faces)==0:
            cv2.putText(blurred_frame, 'please look into camera', (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
        for x, y, w, h in faces:
            detected_face = frame[int(y):int(y+h), int(x):int(x+w)]
            blurred_frame[y:y+h, x:x+w] = detected_face
            cv2.putText(blurred_frame, 'Press b to capture portrait image', (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,0), 2)
            if pressedKey == ord('b'):
                cv2.imwrite('Blur.jpg',blurred_frame)
        cv2.imshow('cam',blurred_frame)
        print(detected_face)

        if cv2.waitKey(1) == ord('x'):
            break

    return main()

In [16]:
cap=cv2.VideoCapture(0)
main()              #code for starting the camera